# SUALBSP-I: Parser und Gurobi-Modell
Dieses Notebook lädt `.alb`-Instanzen und baut ein Gurobi-Modell zur Minimierung der Stationsanzahl (Type-I).
Es nutzt die in den Dataset-Dateien vorhandenen Vorwärts- und Rückwärts-Rüstzeiten.

In [1]:
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Iterable, List, Optional, Tuple

import gurobipy as gp
from gurobipy import GRB


In [2]:
@dataclass
class SUALBInstance:
    n_tasks: int
    cycle_time: int
    task_times: Dict[int, int]
    precedences: List[Tuple[int, int]]
    setup_forward: Dict[Tuple[int, int], int]
    setup_backward: Dict[Tuple[int, int], int]


def _parse_section(lines: Iterable[str]) -> Dict[str, List[str]]:
    sections: Dict[str, List[str]] = {}
    current = None
    for raw in lines:
        line = raw.strip()
        if not line:
            continue
        if line.startswith("<") and line.endswith(">"):
            current = line.strip("<>").strip()
            sections[current] = []
            continue
        if current is None:
            continue
        sections[current].append(line)
    return sections


def parse_alb_file(path: Path) -> SUALBInstance:

    sections = _parse_section(path.read_text().splitlines())

    n_tasks = int(sections["number of tasks"][0])
    cycle_time = int(sections["cycle time"][0])

    task_times: Dict[int, int] = {}
    for entry in sections["task times"]:
        task, duration = entry.split()
        task_times[int(task)] = int(duration)

    precedences: List[Tuple[int, int]] = []
    for entry in sections["precedence relations"]:
        i, j = entry.split(",")
        precedences.append((int(i), int(j)))

    setup_forward: Dict[Tuple[int, int], int] = {}
    for entry in sections.get("setup times forward", []):
        i, rest = entry.split(",")
        j, t = rest.split(":")
        setup_forward[(int(i), int(j))] = int(t)

    setup_backward: Dict[Tuple[int, int], int] = {}
    for entry in sections.get("setup times backward", []):
        i, rest = entry.split(",")
        j, t = rest.split(":")
        setup_backward[(int(i), int(j))] = int(t)

    return SUALBInstance(
        n_tasks=n_tasks,
        cycle_time=cycle_time,
        task_times=task_times,
        precedences=precedences,
        setup_forward=setup_forward,
        setup_backward=setup_backward,
    )


In [3]:
# Beispiel: Instanz parsen (ohne Optimierung)
instance_path = Path('DataSets/DataSet SBF1/SBF1-0.25/Arc111_c=10027.alb')
instance = parse_alb_file(instance_path)
print(f'Aufgaben: {instance.n_tasks}, Cycle time: {instance.cycle_time}')
print(f'Task-Times: {len(instance.task_times)} Einträge')
print(f'Präzedenzkanten: {len(instance.precedences)}')
print(f'Forward-Setups: {len(instance.setup_forward)}, Backward-Setups: {len(instance.setup_backward)}')

# optional: variable to limit number of stations in the model
max_stations = None

Aufgaben: 111, Cycle time: 10027
Task-Times: 111 Einträge
Präzedenzkanten: 176
Forward-Setups: 7440, Backward-Setups: 9856


In [ ]:
tasks = list(instance.task_times.keys()); n = instance.n_tasks; cycle = instance.cycle_time
ub_stations = max_stations or n; stations = list(range(1, ub_stations + 1))

model = gp.Model("SUALBSP-I")

x = model.addVars(tasks, stations, vtype=GRB.BINARY, name="x")
u = model.addVars(stations, vtype=GRB.BINARY, name="u")
z = model.addVars(tasks, tasks, stations, vtype=GRB.BINARY, name="z")
f = model.addVars(tasks, stations, vtype=GRB.BINARY, name="first")
l = model.addVars(tasks, stations, vtype=GRB.BINARY, name="last")
b = model.addVars(tasks, tasks, stations, vtype=GRB.BINARY, name="backward")
p = model.addVars(tasks, stations, vtype=GRB.INTEGER, lb=1, ub=n, name="pos")

model.addConstrs((x.sum(i, "*") == 1 for i in tasks), name="assign_once")
model.addConstrs((x.sum("*", k) <= n * u[k] for k in stations), name="use_station")

model.addConstrs((f.sum("*", k) == u[k] for k in stations), name="one_first_per_station")
model.addConstrs((l.sum("*", k) == u[k] for k in stations), name="one_last_per_station")
model.addConstrs((f[i, k] <= x[i, k] for i in tasks for k in stations), name="first_only_if_assigned")
model.addConstrs((l[i, k] <= x[i, k] for i in tasks for k in stations), name="last_only_if_assigned")

model.addConstrs((z[i, j, k] <= x[i, k] for i in tasks for j in tasks for k in stations), name="order_only_if_assigned_i")
model.addConstrs((z[i, j, k] <= x[j, k] for i in tasks for j in tasks for k in stations), name="order_only_if_assigned_j")
model.addConstrs((z[i, j, k] + z[j, i, k] == x[i, k] + x[j, k] - (1 - u[k]) for i in tasks for j in tasks if i < j for k in stations), name="exact_order_if_both_on_station")

big_m = n
model.addConstrs((p[i, k] - p[j, k] + big_m * z[i, j, k] <= big_m - 1 + (1 - x[i, k]) * big_m + (1 - x[j, k]) * big_m for i in tasks for j in tasks if i != j for k in stations), name="mtz_order")

model.addConstrs((z[i, j, k] >= f[i, k] + x[j, k] - 1 for i in tasks for j in tasks if i != j for k in stations), name="first_precedes_all")
model.addConstrs((z[i, j, k] <= 1 - l[j, k] + x[i, k] for i in tasks for j in tasks if i != j for k in stations), name="last_followed_by_none")

model.addConstrs((b[i, j, k] <= l[i, k] for i in tasks for j in tasks for k in stations), name="backward_last")
model.addConstrs((b[i, j, k] <= f[j, k] for i in tasks for j in tasks for k in stations), name="backward_first")
model.addConstrs((b[i, j, k] >= l[i, k] + f[j, k] - 1 for i in tasks for j in tasks for k in stations), name="backward_iff")

station_index = {i: gp.quicksum(k * x[i, k] for k in stations) for i in tasks}
model.addConstrs((station_index[i] <= station_index[j] for i, j in instance.precedences), name="precede_station")
model.addConstrs((z[i, j, k] >= x[i, k] + x[j, k] - 1 for i, j in instance.precedences for k in stations), name="precede_order_same_station")

forward_default = 0; backward_default = 0
for k in stations:
    processing = gp.quicksum(instance.task_times[i] * x[i, k] for i in tasks)
    forward_setups = gp.quicksum(instance.setup_forward.get((i, j), forward_default) * z[i, j, k] for i in tasks for j in tasks if i != j)
    backward_setups = gp.quicksum(instance.setup_backward.get((i, j), backward_default) * b[i, j, k] for i in tasks for j in tasks)
    model.addConstr(processing + forward_setups + backward_setups <= cycle * u[k], name=f"cycle_{k}")

model.setObjective(gp.quicksum(u[k] for k in stations), GRB.MINIMIZE)


Start building model...
12321
12321
 -> variables done
Constraints 1 (assignment)...
 -> done
Constraints 2 (station usage and ordering)...
 -> done


KeyboardInterrupt: 

In [ ]:
model.update()
model

<gurobi.Model Continuous instance SUALBSP-I: 0 constrs, 0 vars, Parameter changes: Username=(user-defined), LicenseID=2732782>

In [ ]:
model.optimize()

In [5]:
import matplotlib.pyplot as plt

# -------------------------------
# 1) Tasks je Station sammeln
# -------------------------------
station_tasks = {k: [] for k in stations}

for i in tasks:
    for k in stations:
        if x[i, k].X > 0.5:
            station_tasks[k].append(i)


# -------------------------------
# 2) Reihenfolge je Station herstellen (über z-Variablen)
# -------------------------------
ordered_station_tasks = {}

for k in stations:
    tasks_here = station_tasks[k]

    if not tasks_here:
        ordered_station_tasks[k] = []
        continue

    # finde die erste Aufgabe (hat keinen Vorgänger)
    first = None
    for i in tasks_here:
        has_pred = any(z[j, i, k].X > 0.5 for j in tasks_here if j != i)
        if not has_pred:
            first = i
            break

    order = [first]
    current = first

    # gehe der Kette entlang: current -> next
    while True:
        next_tasks = [j for j in tasks_here if z[current, j, k].X > 0.5]
        if not next_tasks:
            break
        current = next_tasks[0]
        order.append(current)

    ordered_station_tasks[k] = order


# -------------------------------
# 3) Plot – Processing + Forward + Backward setups
# -------------------------------
fig, ax = plt.subplots(figsize=(12, 0.6 * len(stations) + 2))

y_positions = []
y_labels = []

for row, k in enumerate(sorted(ordered_station_tasks.keys())):
    tasks_k = ordered_station_tasks[k]

    y_positions.append(row)
    y_labels.append(f"Station {k}")

    start_time = 0

    for idx, op in enumerate(tasks_k):

        # --- Processing block ---
        duration = t[op]
        ax.barh(row, duration, left=start_time, color="steelblue",
                label="processing" if (row == 0 and idx == 0) else None)

        ax.text(start_time + duration/2, row,
                f"Op {op}", ha="center", va="center",
                color="white", fontsize=9)

        start_time += duration

        # --- Forward setup (if there is a next task) ---
        if idx < len(tasks_k) - 1:
            nxt = tasks_k[idx + 1]
            s = setup_forward.get((op, nxt), 0)

            if s > 0:
                ax.barh(row, s, left=start_time, color="gray", alpha=0.6,
                        label="forward setup" if (row == 0 and idx == 0) else None)
                start_time += s

    # --- Backward setup (last -> first) ---
    if tasks_k:
        last_op = tasks_k[-1]
        first_op = tasks_k[0]
        sb = setup_backward.get((last_op, first_op), 0)

        if sb > 0:
            ax.barh(row, sb, left=start_time, color="black", alpha=0.5,
                    label="backward setup" if row == 0 else None)
            start_time += sb


# -------------------------------
# 4) Zykluszeit markieren
# -------------------------------
ax.axvline(C.X if hasattr(C, "X") else C, linestyle="--", linewidth=1, color="red")
ax.text((C.X if hasattr(C, "X") else C), -0.7,
        f"C = {(C.X if hasattr(C, 'X') else C):.2f}",
        ha="center", va="bottom", fontsize=10, color="red")


# -------------------------------
# 5) Layout & Achsen
# -------------------------------
ax.set_yticks(y_positions)
ax.set_yticklabels(y_labels)
ax.set_xlabel("Time")
ax.set_title("SUALBSP — Station Workload (Processing + Setups)")
ax.invert_yaxis()

handles, labels = ax.get_legend_handles_labels()
if handles:
    ax.legend()

plt.tight_layout()
plt.show()


AttributeError: Index out of range for attribute 'X'